In [16]:
import glob

path = "../api/index_documents"

files  = glob.glob(path + "/*")

#for filename in glob.glob(f"{path}/*"):
#    print(filename)

files

['../api/index_documents\\ED2_PAPER+ID+05+GENERATIVE+ART.pdf',
 '../api/index_documents\\UH Onboarding Manual.pdf']

In [17]:
from langchain_community.document_loaders import PyPDFLoader

docs = []
for file in files:
    loader = PyPDFLoader(file)
    pages = loader.load()
    docs.append(pages)
    print(file)

../api/index_documents\ED2_PAPER+ID+05+GENERATIVE+ART.pdf
../api/index_documents\UH Onboarding Manual.pdf


In [7]:
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.documents import Document
import os

embeddings = AzureOpenAIEmbeddings(
                azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                api_key=os.getenv('AZURE_OPENAI_KEY'),
                azure_deployment=os.getenv('EMBEDDINGS_DEPLOYMENT_NAME')
            )



#vector_store = FAISS.from_documents(docs[0], embeddings)

#vector_store.save_local(os.path.join(full_path, "faiss_index"))
#index_path = os.path.join(full_path, "faiss_index")

In [36]:
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss

d = 1536
faiss_index = faiss.IndexFlatL2(d)

vs = FAISS( 
    embedding_function=embeddings,
    index=faiss_index,
    docstore= InMemoryDocstore(),
    index_to_docstore_id={}
    )

In [37]:
vs.add_documents(docs[0])

['150c06d2-c2e2-4606-b532-f2d23d9b8a69',
 'f9a3bcd5-8cb8-44e3-b608-4c98296e073f',
 '527f2c0c-b259-49dc-8508-9aa38a698674',
 '5f87d2b1-b797-4e96-822c-0dd3e08603a9',
 '1dba8287-2d01-4383-84ed-fec70cd962f8',
 '476f9526-aa71-48e9-bc16-df5a9980763e',
 '52aae229-4cbf-4fd5-bfa4-bea8ac2f55d7',
 '6123d1dc-99ef-4ab5-bdf1-9400f813af63',
 'b3b1e494-4f60-4adb-8c05-d390e5b19865',
 '51cf03b7-da8a-4047-b076-1ad8ce488ef3',
 '35afd121-e91c-422b-8e3b-bad72cb871cc',
 '66c101ef-485b-4ccd-bd36-e21178638350']

In [30]:
vector_store.similarity_search("genai", k=3)

[Document(id='98c139b5-e527-497c-a3dd-eabc9c2f63c0', metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-12-15T20:26:26+08:00', 'author': 'Teddy Soikun', 'moddate': '2023-12-15T20:26:26+08:00', 'source': '../api/index_documents\\ED2_PAPER+ID+05+GENERATIVE+ART.pdf', 'total_pages': 12, 'page': 11, 'page_label': '12'}, page_content='[69] \n \nREFERENCES \n \nA., A. (2023). The Renaissance of Creativity: Exploring the Impact of Generative AI in the Creative \nIndustry. https://www.linkedin.com/pulse/renaissance -creativity-exploring-impact-generative-ai-\nadam \nAela, E. (2023, April 1). Artificial Intelligence: How AI is Changing Art. \nhttps://aelaschool.com/en/art/artificial-intelligence-art-changes/ \nAhmed, R. (2023). The Future of Art and Creativity in the Age of AI. \nhttps://www.linkedin.com/pulse/future-art-creativity-age-ai-rohan-ahmed \nGwafan, D. (2023). Python Programming Language: A Versatile and Powerful Tool for Modern \nDe

In [19]:
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss
parent_dir = "../api/index"

def create_vector_store(documents):

        index = faiss.IndexFlatL2(1536)

        vector_store = FAISS(
            embedding_function=embeddings,
            index=index,
            docstore= InMemoryDocstore(),
            index_to_docstore_id={}
        )

        for document in documents:
            vector_store.add_documents(document)

        vector_store.save_local(os.path.join(parent_dir, "faiss_index"))
        index_path = os.path.join(parent_dir, "faiss_index")
        return index_path

In [20]:
create_vector_store(docs)

'../api/index\\faiss_index'

In [11]:
index_path = "../api/index/faiss_index"
from langchain_community.vectorstores import FAISS

def query_index(query: str):
    vector_store = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
    #retriever = vector_store.as_retriever()
    #documents = retriever.invoke(query)
    #result_string = "\n\n".join(doc.page_content for doc in documents)
    return vector_store

In [26]:
os.getcwd()+"\\index_documents"

'c:\\Users\\p.a.rodriguez.canedo\\Documents\\Hackaton_2025\\Notebooks\\index_documents'

In [21]:
vs = query_index("generative ai in art")

In [23]:
vs.similarity_search("generative ai")

[Document(id='8a061273-52b8-4f20-a1a5-0dd12f5390aa', metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-12-15T20:26:26+08:00', 'author': 'Teddy Soikun', 'moddate': '2023-12-15T20:26:26+08:00', 'source': '../api/index_documents\\ED2_PAPER+ID+05+GENERATIVE+ART.pdf', 'total_pages': 12, 'page': 1, 'page_label': '2'}, page_content="[59] \n \nproduce more advanced and higher-quality work and has helped visual artists produce art. AI \nis modernizing some processes, but it still requires human involvement to create something \nuseful and, most importantly, interesting. Therefore, AI can be seen as a tool that complements \nand enhances the work of designers and artists to make their work more productive and efficient. \nDesigners can fill the gap between human touch and creativity, which is fundamental to the \nartwork. Unlike computers, artists are endowe d with their own emotions and unique \nperspectives (Nigeria, 2023). \nGenerative art,

In [28]:
vs.index.ntotal

30